# TorchMD API tutorial

## System setup

We use the `moleculekit` library for reading the input topologies and starting coordinates

In [1]:
from moleculekit.molecule import Molecule
import os

testdir = "../tests/data/prod_alanine_dipeptide_amber/"
mol = Molecule(os.path.join(testdir, "structure.prmtop"))  # Reading the system topology
mol.read(os.path.join(testdir, "input.coor"))  # Reading the initial simulation coordinates
mol.read(os.path.join(testdir, "input.xsc"))  # Reading the box dimensions

Next we will load a forcefield file and use the above topology to extract the relevant parameters which will be used for the simulation

In [2]:
from torchmd.forcefields.forcefield import ForceField
from torchmd.parameters import Parameters
import torch

precision = torch.float
device = "cuda:0"

ff = ForceField.create(mol, os.path.join(testdir, "structure.prmtop"))
parameters = Parameters(ff, mol, precision=precision, device=device)

Now we can create a `System` object which will contain the state of the system during the simulation, including:
1. The current atom coordinates
1. The current box size
1. The current atom velocities
1. The current atom forces

In [3]:
from torchmd.integrator import maxwell_boltzmann
from torchmd.systems import System

system = System(mol.numAtoms, nreplicas=1, precision=precision, device=device)
system.set_positions(mol.coords)
system.set_box(mol.box)
system.set_velocities(maxwell_boltzmann(parameters.masses, T=300, replicas=1))

Lastly we will create a `Force` object which will be used to evaluate the potential on a given `System` state

In [4]:
from torchmd.forces import Forces

forces = Forces(parameters, cutoff=9, rfa=True, switch_dist=7.5, terms=["bonds", "angles", "dihedrals", "impropers", "1-4", "electrostatics", "lj"])
# Evaluate current energy and forces. Forces are modified in-place
Epot = forces.compute(system.pos, system.box, system.forces, returnDetails=True)

print(Epot)
print(system.forces)

[{'electrostatics': -2568.498046875, 'lj': 359.2510986328125, 'bonds': 3.957749366760254, 'angles': 2.8445725440979004, 'dihedrals': 10.57987117767334, '1-4': 0.0, 'impropers': 1.2417081594467163, 'external': 0.0}]
tensor([[[  3.0404,   1.7028,   3.8141],
         [-15.2398, -17.4599,   5.3314],
         [  2.5749,   3.8611,  -4.1888],
         ...,
         [-22.4462,   8.8784,  32.4493],
         [  1.1741,  -8.0141, -15.6699],
         [ 20.2039,  -3.2618, -10.9875]]], device='cuda:0')


## Dynamics

For performing the dynamics we will create an `Integrator` object for integrating the time steps of the simulation as well as a `Wrapper` object for wrapping the system coordinates within the periodic cell

In [5]:
from torchmd.integrator import Integrator
from torchmd.wrapper import Wrapper

langevin_temperature = 300  # K
langevin_gamma = 0.1
timestep = 1  # fs

integrator = Integrator(system, forces, timestep, device, gamma=langevin_gamma, T=langevin_temperature)
wrapper = Wrapper(mol.numAtoms, mol.bonds if len(mol.bonds) else None, device)

In [6]:
from torchmd.minimizers import minimize_bfgs

minimize_bfgs(system, forces, steps=500)  # Minimize the system

Iter  Epot            fmax    
   0   -2190.623047    64.694480
   1   -1917.852238    153.854096
   2   -2307.939932    37.247708
   3   -2360.717695    24.187763
   4   -2389.793893    18.115493
   5   -2406.956225    13.061252
   6   -2453.368793    17.315585
   7   -2481.079855    87.193074
   8   -2511.369712    30.957611
   9   -2522.752202    25.994617
  10   -2530.703532    12.665408
  11   -2540.424319    23.026250
  12   -2553.543909    37.351916
  13   -2570.212033    33.363623
  14   -2555.531865    92.334061
  15   -2578.095990    38.643008
  16   -2589.265238    15.870475
  17   -2594.450592    16.900713
  18   -2600.693053    19.472768
  19   -2609.079230    32.284856
  20   -2617.942266    17.985055
  21   -2622.965751    16.227422
  22   -2629.238345    15.246683
  23   -2632.595876    20.107341
  24   -2638.856846    19.588351
  25   -2645.886016    41.934652
  26   -2652.045023    27.116244
  27   -2654.633144    19.017412
  28   -2658.484353    9.440699
  29   -2663

 255   -2819.876273    2.754193
 256   -2819.965905    2.362493
 257   -2820.090707    4.096729
 258   -2820.224468    5.841139
 259   -2820.446377    4.065695
 260   -2820.581072    3.096381
 261   -2820.710392    2.591622
 262   -2820.875677    3.333195
 263   -2820.990844    5.364328
 264   -2821.172486    2.786425
 265   -2821.302439    2.028732
 266   -2821.416199    2.977870
 267   -2821.547590    14.722537
 268   -2821.775137    3.508366
 269   -2821.885494    2.659619
 270   -2822.019893    3.910247
 271   -2822.184173    4.758809
 272   -2822.294496    3.754042
 273   -2822.454810    2.532179
 274   -2822.552983    2.604264
 275   -2822.687375    4.216683
 276   -2822.790494    7.431732
 277   -2822.991082    3.731536
 278   -2823.072455    2.083044
 279   -2823.141224    2.092084
 280   -2823.258758    3.983892
 281   -2823.296825    9.486460
 282   -2823.554481    3.620488
 283   -2823.647153    1.987778
 284   -2823.734698    1.744476
 285   -2823.838665    2.617844
 286   

Create a CSV file logger for the simulation which keeps track of the energies and temperature.

In [7]:
from torchmd.utils import LogWriter

logger = LogWriter(path="logs/", keys=('iter','ns','epot','ekin','etot','T'), name='monitor.csv')

Writing logs to  logs/monitor.csv


Now we can finally perform the full dynamics

In [8]:
from tqdm import tqdm 
import numpy as np

FS2NS = 1E-6 # Femtosecond to nanosecond conversion

steps = 1000
output_period = 10
save_period = 100
traj = []

trajectoryout = "mytrajectory.npy"

iterator = tqdm(range(1, int(steps / output_period) + 1))
Epot = forces.compute(system.pos, system.box, system.forces)
for i in iterator:
    Ekin, Epot, T = integrator.step(niter=output_period)
    wrapper.wrap(system.pos, system.box)
    currpos = system.pos.detach().cpu().numpy().copy()
    traj.append(currpos)
    
    if (i*output_period) % save_period  == 0:
        np.save(trajectoryout, np.stack(traj, axis=2))

    logger.write_row({'iter':i*output_period,'ns':FS2NS*i*output_period*timestep,'epot':Epot,'ekin':Ekin,'etot':Epot+Ekin,'T':T})

100%|██████████| 100/100 [00:31<00:00,  3.13it/s]
